# Design Pattern 14 - Distribution Strategy (Chapter 4 - Model Training Patterns)

## Intro

### Distribution strategy abstract definition
Distribution strategy is the use of multiple devices either on the same machine, or across different machines to both overcome hardware limitations and speed up training for large models and data

### Problem / Motivation
1. "it’s been shown that increasing the scale of deep learning, with respect to the number of training examples, the number of model parameters, or both leads to a significant increase in performance"
![](./Images/Compute_related_to_performance.PNG) |
1. State of the art models can take enourmous amounts of time to train
    1. Makes it hard to develop models, since results are unknown for a long time
    1. Can increase the cost while using time based cloud compute resources, especially in the "paying for a compute service" model


### Solution Overview 

As the title and definition hint: The solution is to split the effort of training the model across multiple machines

Achieved under two different ways: Model Paralellism and Data Parallelism

Model parallelism             |  Data parallelism
:-------------------------:|:-------------------------:
![](./Images/Model_parallelism.png)  |  ![](./Images/Data_parallelism.png)

As the images may suggest, Model parallelism is splitting up the training by seperating model units across devices, and Data parallelism is splitting up the training by sending different data to *the same\** model across devices. Devices must communicate to eachother to take and pass on their work.

\*To be explored

#### Challenges

In any case for a distribution strategy, the main challenge is minimizing the time for device communication. Optimized file formats, prefetching communication data on the primary node to be served (or other ways to minimize resource idle time) can aid the issue.


## Data Parallelism

Data Parallelism can come in two forms: Synchronous - where at each step in model training, all nodes are updated to have the same model, and Asynchronous - All nodes communicate the the primary node in their own time to retrieve model weights and communicate results

### Synchronous

The workers train on different slices of input data in parallel and the gradient values are aggregated at the end of each training step. This is performed via an all-reduce algorithm

![All Reduce Example](https://mpitutorial.com/tutorials/mpi-reduce-and-allreduce/mpi_allreduce_1.png)

All-reduce will wait for all nodes to compute the gradient of the loss at each weight and average these gradients together for a single gradient at each weight with which to update the weight with. A copy of the new model is then sent back to all nodes for another step in training.

The larger the model, the more gradients to communicate to the primary server, and the more weights to communicate back to the worker nodes. 

Some method of reducing the overhead of the I/O is to let the nodes communicate with eachother for faster propagation of results (Shown in the Data parallelism figure above). But overall, choosing a strategy that most suits available hardware can differ across infrastructure.

Below, are the 3 strategies offered by tensorflow: `MultiWorkerMirrored` copies data across all nodes. `MirroredStrategy` Copies data on each GPU in a machine, and `CentralStorageStrategy` has the CPU communicate with each GPU without GPU-GPU information propagation

![tf_ddp_strat_table](./Images/tensorflow_ddp_stratselection.PNG)

The equivalent in PyTorch depends on the users own implementation of serving the data, ranging from strategies similar to CentralStorage to MultiWorkerMirroredStrategy with a CLI launch of PyTorch distributed specifying the master-node for model storage and all-reduce

### Asynchronous

![Parameter Server Arch](./Images/AsynParamServer.PNG)

Models Weights and training slices are updated asyncronously typically with a Parameter-Server architecture: "data and workloads are distributed over worker nodes, while the server nodes maintain globally shared parameters".

The server node in this case does not perform the all-reduce operation, but will change the model after a certain interval / number of batches recieved back in order to update the weights of the model.

Workers that fail and require a reboot will stop contributing to the training, but the training will still progress. This method loses determinism of results, and training slices may be entirely lost to a dead server. (Use case for virtual epochs!)

The point of failure then has been reduced to the Server node alone, and worker nodes do not have to idle until remaining nodes finish work for the updated model like in Synchronous training.

Keras implements `ParameterServerStrategy` for out of the box Async training. As before with PyTorch, user defined data serving with the torch helper functions can achieve similar results: [Tutorial](https://pytorch.org/tutorials/intermediate/rpc_param_server_tutorial.html)

### ASync vs Sync

Synchronous Data Parallelism             |  Asynchronous Data Parallelism
:--------------------------------:|:--------------------------------:
✔️ Increases Data Throughput | ✔️ Increases Data Throughput
✔️ Faster Model Training Times | ✔️ Faster Model Training Times
✔️ Deterministic  |  ❌ Non-Deterministic
✔️ All work captured/utilized | ❌ Probability for work done on stale weights
❌ Multiple Single Points of Failure | ✔️ High Failure Mitigation
❌ Resource Idle Time | ✔️ Only I/O Speed Limited
❌ I/O bottleneck | ❌ I/O bottleneck

## Model Parallelism

Partitioning parts of a network and their associated computations across multiple cores, the computation and memory workload is distributed across multiple devices.

The I/O then happens between neurons of a network, hopefully the diagrams do well to explain this (Bold connctions must use I/O to communicate results to the necessary machines).
![](./Images/Model_parallelism.png)


## Extra Items of Note

1. The solution, or which strategy to choose always depends on the hardware. Model Parallelism is more typical in model inference on small device scenarious, Async training on unreliable devices, Syncronous training for reproducible large model development etc. 
1. Custom hardware can assist in minimizing I/O waits: e.g. TPUs are optimized to communicate between eachother
1. Batch size limit: a batch size too high misses convergence (see below)
1. Sometimes performing both model and data parallelism is required/desired. [Mesh Tensorflow](https://arxiv.org/abs/1811.02084) attempts to provide a framework to tackle this issue.

## Proof of concept results from the books

![](./Images/more_distrib_workers_increases_throughput.PNG)
![](./Images/more_workers_faster_convergence_distrib.PNG)
![](./Images/Batch_size_too_large_causes_issues.PNG)


## Code snippets from the book

Probably due to the "It depends" nature of the avaiable hardware matching the distribution strategy, no concrete implementation of distributed strategy in practice for training is implemented, even on the github repository. Instead compiled are the code examples which point to ML framework's out of the box strategies for discussed strategies.

Not mentioned but worth considering are: [Ray](https://docs.ray.io/en/latest/train/train.html), [DeepSpeed](https://www.deepspeed.ai/), or [Torch Lightning](https://pytorch-lightning.readthedocs.io/en/stable/accelerators/gpu_intermediate.html) . Other parallel ML frameworks are available.

In [ ]:
# mirrored strategy tensorflow snippet

# Simply replacing the strategy with another found in the docs 
# https://www.tensorflow.org/api_docs/python/tf/distribute 
# allows flexible training selection
def tf_snippet():
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        model = tf.keras.Sequential([tf.keras.layers.Dense(32, input_shape=(5,)),
                                     tf.keras.layers.Dense(16, activation='relu'),
                                     tf.keras.layers.Dense(1)])
        model.compile(loss='mse', optimizer='sgd')

PyTorch script from shell example
```
python -m torch.distributed.launch --nproc_per_node=4 \
       --nnodes=16 --node_rank=3 --master_addr="192.168.0.1" \
       --master_port=1234 my_pytorch.py
```

In [ ]:
# Torch important components snippets

# Docs: https://pytorch.org/docs/stable/distributed.html


def torch_snippets():
    torch.distributed.init_process_group(backend="nccl") # different backends per different infrastructure
    local_rank = _ # local rank = rank of the distributed machine in machine hierarchy (read from script params)
    device = torch.device("cuda:{}".format(local_rank)) 
    model = _ # Some torch model
    model = model.to(device)
    ddp_model = DistributedDataParallel(model, device_ids=[local_rank],
                                            output_device=local_rank)


    sampler = DistributedSampler(dataset=trainds)
    loader = DataLoader(dataset=trainds, batch_size=batch_size,
                        sampler=sampler, num_workers=4)
    
    ...
    
    for data in train_loader:
        features, labels = data[0].to(device), data[1].to(device)

In [ ]:
# tensorflow TPU clusterresolver snippets

# Docs: https://www.tensorflow.org/api_docs/python/tf/distribute/cluster_resolver/ClusterResolver
def tf_TPU_snippets():
    cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
        tpu=tpu_address)
    tf.config.experimental_connect_to_cluster(cluster_resolver)
    tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(cluster_resolver)

## Real world examples

Data Distribution on SPICE: Failed, see ./dp14_demo/ 
Challenges: 
- The cluster must know the location of all devices (Sbatch jobs go to variable addresses)
- Hard to debug networking issues on spice
- lack of implementation examples from the book

## Further Reading

- Model paralellism example paper: https://arxiv.org/pdf/1907.05019.pdf
- Parameter Server Paper: http://web.eecs.umich.edu/~mosharaf/Readings/Parameter-Server.pdf
- Online MLDP Book Chapter: https://learning.oreilly.com/library/view/machine-learning-design/9781098115777/ch04.html#why_it_works-id00313
- ML Framework docs on model: https://docs.chainer.org/en/stable/chainermn/model_parallel/overview.html
- Torch distributed insight paper: https://arxiv.org/pdf/2006.15704.pdf